In [3]:
from qiskit_cartan.cartan_plugin import *

import logging
logging.basicConfig(level='DEBUG')

#### Check `synth_cartan` works as intended with an example Heisenberg Hamiltonian

In [4]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
test_heisenberg_ham = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], coeffs = [1,1,1,1,1,1])
test_time_evolve = 3
test_paulievolutiongate = PauliEvolutionGate(test_heisenberg_ham, time=test_time_evolve)
qc = synth_cartan(test_paulievolutiongate, 0)
print(qc)

(3, 0, 3)
[(0, 2, 2), (2, 2, 0), (2, 0, 2)]
Optimization terminated successfully.
         Current function value: -2.586419
         Iterations: 20
         Function evaluations: 25
         Gradient evaluations: 25
--- 0.08607029914855957 seconds ---
Optimization Error:
4.834370506759767e-15
     ┌───────────────────────┐┌────────────────────────┐»
q_0: ┤0                      ├┤0                       ├»
     │                       ││                        │»
q_1: ┤1 exp(-it XYZ)(1.1781) ├┤1 exp(-it XZY)(0.46588) ├»
     │                       ││                        │»
q_2: ┤2                      ├┤2                       ├»
     └───────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                       ├┤0                        ├»
«     │                        ││                         │»
«q_1: ┤1 exp(-it ZYX)(0.47512) ├┤1 exp(-it YZX)(-0.30774) ├»
«     │                        ││                      

In [5]:
from qiskit import Aer, transpile

unitary_simulator = Aer.get_backend('unitary_simulator')
qc_transp = transpile(qc, unitary_simulator)
final_unitary_cartan = unitary_simulator.run(qc_transp).result().get_unitary()

import scipy
from qiskit.quantum_info import SparsePauliOp

H = SparsePauliOp(["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"], 
                  np.array([1, 1, 1, 1, 1, 1])).to_matrix()
propagator = scipy.linalg.expm(-1j*H*test_time_evolve)
matrix_norm = np.linalg.norm(propagator - final_unitary_cartan)
print("\n")
print("matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: \n", matrix_norm)

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.00000 (ms)
DEBUG:stev



matrix norm of the difference between the ideal propagator and that obtained by cartan decomposition: 
 2.704959253147437e-08


#### Check our plugin is installed

In [6]:
from qiskit.transpiler.passes.synthesis.plugin import HighLevelSynthesisPluginManager
HLS_plugin_manager = HighLevelSynthesisPluginManager()
print(HLS_plugin_manager.plugins.names())

DEBUG:stevedore.extension:found extension EntryPoint(name='clifford.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisClifford', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='linear_function.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisLinearFunction', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.acg', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:ACGSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.basic', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedo

['clifford.default', 'linear_function.default', 'permutation.acg', 'permutation.basic', 'permutation.default', 'permutation.kms', 'PauliEvolution.cartan2']


#### Test the pass

In [7]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
from qiskit.transpiler import PassManager

qc_before = QuantumCircuit(3)
qc_before.append(test_paulievolutiongate, range(3))
print("before: ")
qc_before.draw()

before: 


┌──────────────────────────────────────────────────┐
q_0: ┤0                                                 ├
     │                                                  │
q_1: ┤1 exp(-it (XXI + YYI + ZZI + IXX + IYY + IZZ))(3) ├
     │                                                  │
q_2: ┤2                                                 ├
     └──────────────────────────────────────────────────┘

In [10]:
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HLSConfig

hls_config = HLSConfig(PauliEvolution=[("cartan2", {"random_seed":0})])
pm = PassManager()
pm.append(HighLevelSynthesis(hls_config=hls_config))
qc_after = pm.run(qc_before)
qc_after.draw()

DEBUG:stevedore.extension:found extension EntryPoint(name='clifford.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisClifford', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='linear_function.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:DefaultSynthesisLinearFunction', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.acg', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:ACGSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.basic', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedore.extension:found extension EntryPoint(name='permutation.default', value='qiskit.transpiler.passes.synthesis.high_level_synthesis:BasicSynthesisPermutation', group='qiskit.synthesis')
DEBUG:stevedo

Running Cartan Synthesis Plugin...
(3, 0, 3)
[(0, 2, 2), (2, 2, 0), (2, 0, 2)]
Optimization terminated successfully.
         Current function value: -2.586419
         Iterations: 20
         Function evaluations: 25
         Gradient evaluations: 25
--- 0.09279322624206543 seconds ---
Optimization Error:
4.834370506759767e-15


┌───────────────────────┐┌────────────────────────┐»
q_0: ┤0                      ├┤0                       ├»
     │                       ││                        │»
q_1: ┤1 exp(-it XYZ)(1.1781) ├┤1 exp(-it XZY)(0.46588) ├»
     │                       ││                        │»
q_2: ┤2                      ├┤2                       ├»
     └───────────────────────┘└────────────────────────┘»
«     ┌────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                       ├┤0                        ├»
«     │                        ││                         │»
«q_1: ┤1 exp(-it ZYX)(0.47512) ├┤1 exp(-it YZX)(-0.30774) ├»
«     │                        ││                         │»
«q_2: ┤2                       ├┤2                        ├»
«     └────────────────────────┘└─────────────────────────┘»
«     ┌─────────────────────────┐┌─────────────────────────┐ ░ »
«q_0: ┤0                        ├┤0                        ├─░─»
«     │                         ││                         │ ░ »
«q_1: ┤1 exp(-it YXZ)(0.059684) ├┤1 exp(-it ZXY)(-0.30425) ├─░─»
«     │                         ││                         │ ░ »
«q_2: ┤2                        ├┤2                        ├─░─»
«     └─────────────────────────┘└─────────────────────────┘ ░ »
«     ┌───────────────────┐┌───────────────────┐┌───────────────────┐ ░ »
«q_0: ┤0                  ├┤0                  ├┤0                  ├─░─»
«     │                   ││                   ││                   │ ░ »
«q_1: ┤1 exp(-it YYI)(-6) ├┤1 exp(-it IYY)(-3) ├┤1 exp(-it YIY)(-3) ├─░─»
«     │                   ││                   ││                   │ ░ »
«q_2: ┤2                  ├┤2                  ├┤2                  ├─░─»
«     └───────────────────┘└───────────────────┘└───────────────────┘ ░ »
«     ┌────────────────────────┐┌──────────────────────────┐»
«q_0: ┤0                       ├┤0                         ├»
«     │                        ││                          │»
«q_1: ┤1 exp(-it ZXY)(0.30425) ├┤1 exp(-it YXZ)(-0.059684) ├»
«     │                        ││                          │»
«q_2: ┤2                       ├┤2                         ├»
«     └────────────────────────┘└──────────────────────────┘»
«     ┌────────────────────────┐┌─────────────────────────┐»
«q_0: ┤0                       ├┤0                        ├»
«     │                        ││                         │»
«q_1: ┤1 exp(-it YZX)(0.30774) ├┤1 exp(-it ZYX)(-0.47512) ├»
«     │                        ││                         │»
«q_2: ┤2                       ├┤2                        ├»
«     └────────────────────────┘└─────────────────────────┘»
«     ┌─────────────────────────┐┌────────────────────────┐
«q_0: ┤0                        ├┤0                       ├
«     │                         ││                        │
«q_1: ┤1 exp(-it XZY)(-0.46588) ├┤1 exp(-it XYZ)(-1.1781) ├
«     │                         ││                        │
«q_2: ┤2                        ├┤2                       ├
«     └─────────────────────────┘└────────────────────────┘